### Prompts

Prompt allow you to create System Messages with input variables, for example, this:
**SystemMessage(content="You are a helpful assistant that translates the English to Spanish.")**
English and Spanish may be dynamic. This can be archieved with templates

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
TEMPLATE = """
You are a helpful assistant that translates the {input_language} to {output_language}
"""

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

prompt_template = PromptTemplate.from_template(
    template=TEMPLATE
)
prompt_template.format(input_language="english", output_language="german")

Passing the input_variables to the constructor will provide additional validation for the template

In [ ]:
prompt_template = PromptTemplate(template=TEMPLATE, input_variables=["input_language", "output_language"])
prompt_template.format(input_language="english", output_language="german")


### Few Shot Prompt - provide a few examples in the template

In [ ]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

To improve performance we can provide examples to increase the quality of the output.

In [ ]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}

Examples:
text: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.
sentiment: positive
subject: BellaVista

text: BellaVista restaurant was alright. The food was decent, but nothing stood out.
sentiment: neutral
subject: BellaVista

text: I was disappointed with BellaVista. The service was slow and the dishes lacked flavor.
sentiment: negative
subject: BellaVista

text: SeoulSavor offered the most authentic Korean flavors I've tasted outside of Seoul. The kimchi was perfectly fermented and spicy.
sentiment: positive
subject: SeoulSavor

text: SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste.
sentiment: neutral
subject: SeoulSavor

text: I didn't enjoy my meal at SeoulSavor. The tteokbokki was too mushy and the service was not attentive.
sentiment: negative
subject: SeoulSavor

text: MunichMeals has the best bratwurst and sauerkraut I've tasted outside of Bavaria. Their beer garden ambiance is truly authentic.
sentiment: positive
subject: MunichMeals

text: MunichMeals was alright. The weisswurst was okay, but I've had better elsewhere.
sentiment: neutral
subject: MunichMeals

text: I was let down by MunichMeals. The potato salad lacked flavor and the staff seemed uninterested.
sentiment: negative
subject: MunichMeals
"""


In [ ]:
prompt_template = PromptTemplate(template=TEMPLATE, input_variables=["input"])
prompt_template.format(input="The MunichDeals experience was just awesome!")

LangChain also provides a FewShotPromptTemplate class, which allows creating the examples more modularized

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
    {
        "text": "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.",
        "response": "sentiment: positive\nsubject: BellaVista"
    },
    {
        "text": "BellaVista restaurant was alright. The food was decent, but nothing stood out.",
        "response": "sentiment: neutral\nsubject: BellaVista"
    },
    ### other examples are left out
]

In [ ]:
new_example = {
    "text": "SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste.",
    "response": "sentiment: neutral\nsubject: SeoulSavor"
}
examples.append(new_example)

In [ ]:
example_prompt = PromptTemplate(input_variables=["text", "response"], template="Text: {text}\n{response}")

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="text: {input}",
    input_variables=["input"]
)

In [ ]:
print(prompt.format(input="The MunichDeals experience was just awesome!"))

### Chain-of-thought Prompting

Instead of just providing examples, we can also provide examples which include the whole thought process of why a review is negative, neutral or positive

In [ ]:
TEMPLATE = """
Interprete the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}

Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable." How does this make you feel about BellaVista?
 It sounds like a positive review for BellaVista.

Based on the positive nature of that statement, how would you format your response?
 { "sentiment": "positive", "subject": "BellaVista" }

Now, think about this: "SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste." Does this give a strong feeling either way?
 Not particularly. It seems like a mix of good and not-so-good elements, so it's neutral.

Given the neutral sentiment, how should this be presented?
 { "sentiment": "neutral", "subject": "SeoulSavor" }

Lastly, ponder on this: "I was let down by MunichMeals. The potato salad lacked flavor and the staff seemed uninterested." What's the overall impression here?
 The statement is expressing disappointment and dissatisfaction.

And if you were to categorize this impression, what would it be?
 { "sentiment": "negative", "subject": "MunichMeals" }
"""

### Serializing prompts

Example is an easy prompt since serializing does not work for the **PipelinePromptTemplate** (yet)

In [ ]:
prompt = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
prompt.save("prompt.yaml")
prompt.save("prompt.json")

In [ ]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompt.yaml")
prompt.format(input="chickens")

In [ ]:
prompt = load_prompt("prompt.json")
prompt.format(input="cows")